In [1]:
data = 'https://archive.ics.uci.edu/static/public/222/bank+marketing.zip'

In [2]:
!wget $data

--2024-10-16 10:11:10--  https://archive.ics.uci.edu/static/public/222/bank+marketing.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘bank+marketing.zip’

bank+marketing.zip      [    <=>             ] 999.85K  1.05MB/s    in 0.9s    

2024-10-16 10:11:12 (1.05 MB/s) - ‘bank+marketing.zip’ saved [1023843]



In [29]:
import pandas as pd

In [30]:
df = pd.read_csv('bank-full.csv', sep=';')

In [31]:
df = df[['age', 'job', 'marital', 'education', 'balance', 'housing',
       'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'y']]

In [32]:
df.y = df.y.replace({'yes': 1, 'no':0})

/tmp/ipykernel_23075/2032539318.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.y = df.y.replace({'yes': 1, 'no':0})


In [33]:
from sklearn.model_selection import train_test_split

In [34]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [35]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [36]:
len(df_train), len(df_val), len(df_test)

(27126, 9042, 9043)

In [37]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [38]:
y_train = df_train.y.values
y_val = df_val.y.values
y_test = df_test.y.values

In [39]:
del df_train['y']
del df_val['y']
del df_test['y']

In [40]:
categorical = ['job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']
numerical = ['age', 'balance', 'day', 'campaign', 'pdays', 'previous']

In [41]:
from sklearn.feature_extraction import DictVectorizer

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [42]:
from sklearn.linear_model import LogisticRegression

In [43]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000)

In [44]:
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, solver='liblinear')

In [45]:
y_pred = model.predict_proba(X_val)[:, 1]

### Question 3

In [46]:
from sklearn.metrics import accuracy_score

In [51]:
actual_positive = (y_val == 1)
actual_negative = (y_val == 0)

In [52]:
t = 0.0
predict_positive = (y_pred >= t)
predict_negative = (y_pred < t)

In [53]:
tp = (predict_positive & actual_positive).sum()
tn = (predict_negative & actual_negative).sum()

In [54]:
fp = (predict_positive & actual_negative).sum()
fn = (predict_negative & actual_positive).sum()

In [55]:
p = tp / (tp + fp)

In [56]:
r = tp / (tp + fn)